In [83]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
import pandas as pd
import numpy as np
from mlfaker.generators import BaseGenerator, NormalGenerator, CategoricalGenerator

In [85]:
g = BaseGenerator(data_name="blah")
g.set_generator("normal", loc=0, scale=1)
g.generate(10)

0    1.624345
1   -0.611756
2   -0.528172
3   -1.072969
4    0.865408
5   -2.301539
6    1.744812
7   -0.761207
8    0.319039
9   -0.249370
Name: blah, dtype: float64

In [86]:
g = NormalGenerator(data_name="blah")
g.generate(10)

0    1.624345
1   -0.611756
2   -0.528172
3   -1.072969
4    0.865408
5   -2.301539
6    1.744812
7   -0.761207
8    0.319039
9   -0.249370
Name: blah, dtype: float64

In [87]:
g = CategoricalGenerator(data_name="blah", classes=["cat", "dog"])
g.generate(10)

0    dog
1    dog
2    cat
3    cat
4    dog
5    dog
6    dog
7    dog
8    dog
9    cat
Name: blah, dtype: object